# Variational Quantum Classifier on EuroSAT Dataset (Fully Quantum with PCA)

This notebook implements a fully quantum variational classifier for Earth Observation data using:
- **Dataset**: EuroSAT (satellite imagery classification)
- **Dimensionality Reduction**: PCA to compress images to quantum-compatible size
- **Framework**: PyTorch Lightning + PennyLane
- **Encoding Options**: Angle Encoding vs Amplitude Encoding
- **Circuit Options**: Strongly Entangling Layers vs Basic Entangling Layers

#  COMPLETE NOTEBOOK GUIDE

##  What does this notebook do?

This notebook implements a **variational quantum classifier** for EuroSAT satellite imagery.

### Pipeline:
```
Satellite Image (64x64x3 RGB)
          ↓
    PCA (dimensionality reduction)
          ↓
    8 or 256 features
          ↓
    Quantum Circuit (encoding + variational layers)
          ↓
    8 expectation values
          ↓
    Classical Linear Layer
          ↓
    10 classes (Annual Crop, Forest, etc.)
```

---

##  Available Configurations

### 1. Encoding Type (how to encode data in the quantum circuit):

**Angle Encoding** (`encoding='angle'`):
- Each feature becomes a rotation angle
- 8 qubits = 8 features needed
-  Simpler and faster
-  Less information for qubit

**Amplitude Encoding** (`encoding='amplitude'`):
- Features become quantum state amplitudes
- 8 qubits = 256 features (2^8)
-  More information compresifd
-  Requires normalization

### 2. Circuit Type (quantum circuit structure):

**Strongly Entangling** (`circuit_type='strongly_entangling'`):
- Every qubit withnected to all others
-  Maximum expressivity
-  More parameters to train

**Basic Entangling** (`circuit_type='basic_entangling'`):
- Only nearest-neighbor withnections
-  Faster to train
-  Less expressive

---

##  How to use the Notesbook

### MODE 1: Single Configuration Training

1. **Chooif the withfiguration** by modifying the CONFIG dictionary:
```python
CONFIG = {
    'encoding': 'angle',  # or 'amplitude'
    'circuit_type': 'strongly_entangling',  # or 'basic_entangling'
    'n_qubits': 8,
    'n_layers': 3,
}
```

2. **Run all cells** up to the ifction "Comparative Analysis"

3. **Results**: one trained model in ~10-20 minuti

### MODE 2: Automatic Comparison (4 withfigurations)

1. **Run ALL cells** including the ifction "Comparative Analysis"

2. The notebook automatically trains:
   - Angle + Strongly Entangling
   - Angle + Basic Entangling
   - Amplitude + Strongly Entangling
   - Amplitude + Basic Entangling

3. **Results**:
   -  Comparative charts
   -  Best withfiguration
   -  Fastest withfiguration
   -  CSV with all results
   - Total time: ~10-20 minutes

---

##  What to Expect

### Expected accuracy:
- Angle encoding: ~60-70%
- Amplitude encoding: ~65-75%

### Training times (single withfiguration):
- Per withfiguration: ~3-5 minutes
- All 4: ~10-20 minutes

### Output:
-  Confusion matrix
-  Classification report
-  Example predictions visualizzate
-  Model saved

---

##  When to Uif This Notesbook

**Uif this notebook (PCA-baifd) if:**
-  You want quick exforiments
-  You have risorif computational limited
-  You want understand i withcepts baif del quantum ML
-  You are exploring different encoding types

**Use the other notebook (CNN-baifd) if:**
-  You want maximum accuracy
-  You have GPU available
-  You want features appreif automatically

---

##  Dataset: EuroSAT

**10 classes di land uif:**
1. AnnualCrop - Annual crops
2. Forest - Forests
3. HerbaceousVegetation - Herbaceous vegetation
4. Highway - Highways
5. Industrial - Industrial areas
6. Pasture - Pastures
7. PermanentCrop - Permanent crops
8. Residential - Residential areas
9. River - Rivers
10. SeaLake - Sea/Lakes

**Characteristics:**
- 27,000 total images
- Resolution: 64×64 pixels
- RGB (3 canali)
- Satellite: Sentinel-2

---

##  Important Notess

1. **Automatic download**: The dataset is downloaded automatically (~90 MB)
2. **PCA**: Reduces 12,288 pixels (64×64×3) a 8 or 256 features
3. **Quantum Circuit**: Uses PennyLane with local simulator
4. **Training**: PyTorch Lightning with automatic early stopping
5. **Reproducibility**: Fixed seed (42) for withsistent results



In [ ]:
# Install required packages
!pip install pennylane pytorch-lightning torchvision scikit-learn matplotlib tqdm -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import zipfile
from pathlib import Path
from PIL import Image
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import pennylane as qml
from pennylane import numpy as pnp
import warnings
import urllib3
warnings.filterwarnings('ignore')

## Configuration
Set the hyforparameters and chooif encoding/circuit type

In [ ]:
# Configuration
CONFIG = {
    'n_qubits': 8,  # Number of qubits (PCA will reduce to this dimension)
    'n_layers': 3,  # Number of variational layers in the quantum circuit
    'encoding': 'angle',  # Options: 'angle' or 'amplitude'
    'circuit_type': 'strongly_entangling',  # Options: 'strongly_entangling' or 'basic_entangling'
    'batch_size': 32,
    'learning_rate': 0.001,
    'max_epochs': 5,
    'n_classes': 10,  # EuroSAT has 10 clasifs
    'pca_components': 8,  # Must match n_qubits for angle encoding, or 2^n_qubits for amplitude
}

# Adjust PCA components baifd on encoding type
if CONFIG['encoding'] == 'amplitude':
    CONFIG['pca_components'] = 2 ** CONFIG['n_qubits']  # Amplitude encoding needs 2^n features
else:
    CONFIG['pca_components'] = CONFIG['n_qubits']  # Angle encoding needs n features

print(f"Configuration: {CONFIG}")

## Data Loading and Preprocessing

Download EuroSAT dataift and apply PCA for dimensionality reduction

In [ ]:
import os
import requests
import zipfile
from pathlib import Path
from PIL import Image
import urllib3

def download_eurosat():
    url = "https://madm.dfki.de/files/sentinel/EuroSAT.zip"
    zip_path = "EuroSAT.zip"
    target_dir = "EuroSAT/2750"

    if not os.path.exists(target_dir):
        print("Downloading EuroSAT RGB dataset...")
        response = requests.get(url, stream=True, verify=False)
        total = int(response.headers.get('content-length', 0))
        with open(zip_path, 'wb') as f:
            downloaded = 0
            for data in response.iter_content(chunk_size=8192):
                f.write(data)
                downloaded += len(data)
                done = int(50 * downloaded / total)
                print(f"\r[{'=' * done}{' ' * (50 - done)}] {downloaded/1e6:.1f}/{total/1e6:.1f} MB", end='')
        print("\n Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall("EuroSAT")
        os.remove(zip_path)
        print("Dataset downloaded and extracted successfully!")
    else:
        print("Dataset already available.")

def load_images(data_dir="EuroSAT/2750", max_per_class=300):
    images, labels, classes = [], [], []
    for class_dir in sorted(Path(data_dir).iterdir()):
        if class_dir.is_dir():
            cls = class_dir.name
            classes.append(cls)
            files = list(class_dir.glob("*.jpg"))[:max_per_class]
            for f in files:
                img = np.array(Image.open(f))
                images.append(img)
                labels.append(cls)
    return np.array(images), np.array(labels), classes

# Run download + load
download_eurosat()
images_raw, labels_raw, class_names = load_images(max_per_class=300)
print(f"Loaded {len(images_raw)} images from {len(class_names)} classes.")

In [ ]:
# Create label mapping
label_to_idx = {cls: idx for idx, cls in enumerate(class_names)}
labels_numeric = np.array([label_to_idx[label] for label in labels_raw])

print(f"Label mapping: {label_to_idx}")
print(f"Number of classes: {len(class_names)}")

# Update withfig with correct number of clasifs
CONFIG['n_classes'] = len(class_names)

In [ ]:
# Visualize some samples
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for i in range(10):
    idx = np.random.randint(len(images_raw))
    axes[i].imshow(images_raw[idx])
    axes[i].set_title(f"{labels_raw[idx]}")
    axes[i].axis('off')

plt.suptitle('Sample Images from EuroSAT Dataset', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Prepare data for PCA by flattening and normalizing images
def prepare_data_for_pca(images, labels, max_samples=5000):
    """
    Flatten and normalize images for PCA.
    Uif a subift for efficiency in PCA fitting.
    """
    # Uif subift for PCA fitting to save memory
    indices = np.random.choice(len(images), min(max_samples, len(images)), replace=False)

    features = []
    iflected_labels = []

    for idx in indices:
        # Normalize image to [0, 1]
        img = images[idx].astype(np.float32) / 255.0
        # Flatten the image: (H, W, C) -> (H*W*C,)
        features.append(img.flatten())
        iflected_labels.append(labels[idx])

    return np.array(features), np.array(iflected_labels)

print("Preparing data for PCA...")
X_pca, y_pca = prepare_data_for_pca(images_raw, labels_numeric)
print(f"Feature shape before PCA: {X_pca.shape}")

In [ ]:
# Fit PCA and scaler
print(f"\nFitting PCA with {CONFIG['pca_components']} components...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_pca)

pca = PCA(n_components=CONFIG['pca_components'])
X_pca_reduced = pca.fit_transform(X_scaled)

print(f"Feature shape after PCA: {X_pca_reduced.shape}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_.sum():.4f}")

In [ ]:
# Create custom dataift that applies PCA transformation
class PCATransformedDataset(Dataset):
    """
    Dataset wrapfor that applies PCA transformation to images.
    """
    def __init__(self, images, labels, pca, scaler):
        self.images = images
        self.labels = labels
        self.pca = pca
        self.scaler = scaler

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
      # Get image and normalize
      img = self.images[idx].astype(np.float32) / 255.0
      label = self.labels[idx]

      # Flatten and apply PCA
      img_flat = img.flatten().reshape(1, -1)
      img_scaled = self.scaler.transform(img_flat)
      img_pca = self.pca.transform(img_scaled)

      return torch.tensor(img_pca[0], dtype=torch.float32), int(label)

# Create PCA-transformed dataift
pca_dataift = PCATransformedDataset(images_raw, labels_numeric, pca, scaler)

# Split into train, validation, and test ifts
train_size = int(0.7 * len(pca_dataift))
val_size = int(0.15 * len(pca_dataift))
test_size = len(pca_dataift) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    pca_dataift, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"\nDataset splits:")
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

## Quantum Circuit Definition

Define the quantum circuits with different encoding and entangling strategies

In [ ]:
# Initialize quantum device
dev = qml.device('default.qubit', wires=CONFIG['n_qubits'])

# Define quantum circuit with angle encoding
@qml.qnode(dev, interface='torch')
def quantum_circuit_angle_encoding(inputs, weights):
    """
    Quantum circuit with angle encoding.
    Each feature is encoded as a rotation angle on each qubit.
    """
    # Angle encoding: encode each feature as RY rotation
    for i in range(CONFIG['n_qubits']):
        qml.RY(inputs[i], wires=i)

    # Variational layers
    if CONFIG['circuit_type'] == 'strongly_entangling':
        # Strongly entangling layers with full withnectivity
        qml.StronglyEntanglingLayers(weights, wires=range(CONFIG['n_qubits']))
    else:
        # Basic entangling layers with nearest-neighbor withnectivity
        qml.BasicEntanglerLayers(weights, wires=range(CONFIG['n_qubits']))

    # Measurement: return expectation values for all qubits
    return [qml.expval(qml.PauliZ(i)) for i in range(CONFIG['n_qubits'])]


@qml.qnode(dev, interface='torch')
def quantum_circuit_amplitude_encoding(inputs, weights):
    """
    Quantum circuit with amplitude encoding.
    Features are encoded as amplitudes of the quantum state.
    Requires 2^n_qubits features (normalized).
    """
    # Amplitude encoding: encode features as quantum state amplitudes
    # Normalize inputs to unit vector
    inputs_normalized = inputs / torch.sqrt(torch.sum(inputs**2) + 1e-8)
    qml.AmplitudeEmbedding(features=inputs_normalized, wires=range(CONFIG['n_qubits']), normalize=True)

    # Variational layers
    if CONFIG['circuit_type'] == 'strongly_entangling':
        qml.StronglyEntanglingLayers(weights, wires=range(CONFIG['n_qubits']))
    else:
        qml.BasicEntanglerLayers(weights, wires=range(CONFIG['n_qubits']))

    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(CONFIG['n_qubits'])]


# Select the appropriate circuit baifd on withfiguration
quantum_circuit = quantum_circuit_angle_encoding if CONFIG['encoding'] == 'angle' else quantum_circuit_amplitude_encoding

print(f"Using {CONFIG['encoding']} encoding with {CONFIG['circuit_type']} circuit")
print(f"Circuit outputs: {CONFIG['n_qubits']} expectation values")

In [ ]:
# Visualize the quantum circuit
print("\nQuantum Circuit Diagram:")
print("="*50)

# Create dummy inputs and weights for visualization
dummy_input = torch.randn(CONFIG['pca_components'])

if CONFIG['circuit_type'] == 'strongly_entangling':
    # StronglyEntanglingLayers shape: (n_layers, n_qubits, 3)
    dummy_weights = torch.randn(CONFIG['n_layers'], CONFIG['n_qubits'], 3)
else:
    # BasicEntanglerLayers shape: (n_layers, n_qubits)
    dummy_weights = torch.randn(CONFIG['n_layers'], CONFIG['n_qubits'])

print(qml.draw(quantum_circuit)(dummy_input, dummy_weights))
print("="*50)

## Quantum Model Definition with PyTorch Lightning

In [ ]:
class QuantumClassifier(pl.LightningModule):
    """
    Fully quantum classifier using PennyLane quantum circuit.
    The model withsists of:
    1. Quantum circuit for feature transformation
    2. Classical linear layer for final classification
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.save_hyperparameters()

        # Initialize quantum weights
        if config['circuit_type'] == 'strongly_entangling':
            # StronglyEntanglingLayers: (n_layers, n_qubits, 3)
            weight_shape = (config['n_layers'], config['n_qubits'], 3)
        else:
            # BasicEntanglerLayers: (n_layers, n_qubits)
            weight_shape = (config['n_layers'], config['n_qubits'])

        # Quantum circuit weights (trainable parameters)
        self.q_weights = nn.Parameter(torch.randn(weight_shape) * 0.1)

        # Classical output layer to map quantum measurements to class probabilities
        self.fc = nn.Linear(config['n_qubits'], config['n_classes'])

        # Loss function
        self.criterion = nn.CrossEntropyLoss()

        # Metrics
        self.train_acc = []
        self.val_acc = []

    def forward(self, x):
      """
      Forward pass through quantum circuit and classical layer.
      """
      x = x.float()  # Force float32
      batch_size = x.shape[0]

      # Process each sample through quantum circuit
      quantum_outputs = []
      for i in range(batch_size):
          q_out = quantum_circuit(x[i].float(), self.q_weights)
          quantum_outputs.append(torch.stack(q_out))

      quantum_outputs = torch.stack(quantum_outputs).float()
      logits = self.fc(quantum_outputs)

      return logits

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)

        # Calculate accuracy
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()

        # Log metrics
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)

        # Calculate accuracy
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()

        # Log metrics
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()

        self.log('test_loss', loss)
        self.log('test_acc', acc)

        return loss

    def configure_optimizers(self):
        """
        Configure optimizer for training.
        Using Adam optimizer with configure_optimizers learning rate.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config['learning_rate'])
        # Optional: learning rate scheduler
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

## Training Setup

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

In [ ]:
# Initialize model
model = QuantumClassifier(CONFIG)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nModel Architecture:")
print(model)
print(f"\nTotal parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

In [ ]:
# Setup callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='./checkpoints',
    filename='quantum-classifier-{epoch:02d}-{val_acc:.4f}',
    monitor='val_acc',
    mode='max',
    save_top_k=3
)

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
    verbose=True
)

# Initialize trainer
trainer = pl.Trainer(
    max_epochs=CONFIG['max_epochs'],
    callbacks=[checkpoint_callback, early_stop_callback],
    accelerator='auto',
    devices=1,
    log_every_n_steps=10,
    enable_progress_bar=True
)

print("Trainer initialized successfully!")

## Training

In [ ]:
# Train the model
print("Starting training...")
print("="*50)
trainer.fit(model, train_loader, val_loader)

## Evaluation

In [ ]:
# Test the model
print("\nEvaluating on test ift...")
test_results = trainer.test(model, test_loader)
print(f"\nTest Results: {test_results}")

In [ ]:
from tqdm.auto import tqdm
# Detailed predictions on test set
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        x, y = batch
        logits = model(x)
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

print("\nClassification Report:")
print(classification_report(all_labels, all_preds))

## Visualization of Results

In [ ]:
def visualize_predictions(dataset, model, pca, scaler, images_raw, labels_numeric, num_samples=8):
    """
    Visualize original images with predictions.
    """
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()

    model.eval()
    indices = np.random.choice(len(dataset), num_samples, replace=False)

    with torch.no_grad():
        for idx, ax in zip(indices, axes):
            # Get original image from raw data
            real_idx = dataset.indices[idx]
            img_raw = images_raw[real_idx]
            label = labels_numeric[real_idx]

            # Get PCA features
            img = img_raw.astype(np.float32) / 255.0
            img_flat = img.flatten().reshape(1, -1)
            img_scaled = scaler.transform(img_flat)
            img_pca = pca.transform(img_scaled)
            img_pca_tensor = torch.tensor(img_pca[0], dtype=torch.float32).unsqueeze(0)

            # Predict
            logits = model(img_pca_tensor)
            pred = torch.argmax(logits, dim=1).item()

            # Display
            ax.imshow(img_raw)
            ax.set_title(f'True: {label}, Pred: {pred}',
                        color='green' if pred == label else 'red')
            ax.axis('off')

    plt.tight_layout()
    plt.show()

visualize_predictions(test_dataset, model, pca, scaler, images_raw, labels_numeric)

##  Comparative Analysis: Different Encodings and Circuits

This ifction automatically trains and compares all combinations of:
- **Encodings**: Angle vs Amplitude
- **Circuits**: Strongly Entangling vs Basic Entangling

Total: 4 different withfigurations

In [ ]:
# Configuration for comparative exforiments
COMPARISON_CONFIG = {
    'n_qubits': 8,
    'n_layers': 3,
    'batch_size': 32,
    'learning_rate': 0.01,
    'max_epochs': 3,  # Reduced for faster comparison
    'n_classes': CONFIG['n_classes'],
}

# All combinations to test
encodings = ['angle', 'amplitude']
circuit_types = ['strongly_entangling', 'basic_entangling']

print("Will test all combinations:")
for enc in encodings:
    for circ in circuit_types:
        print(f"  - {enc} encoding + {circ} circuit")

In [ ]:
import time
import pandas as pd

# Dictionary to store results
comparison_results = []

print("Starting comparative experiments...")
print("="*70)

for encoding in encodings:
    for circuit_type in circuit_types:
        print(f"\n{'='*70}")
        print(f"Training: {encoding.upper()} encoding + {circuit_type.upper()} circuit")
        print(f"{'='*70}\n")

        # Update configuration
        exp_config = COMPARISON_CONFIG.copy()
        exp_config['encoding'] = encoding
        exp_config['circuit_type'] = circuit_type

        # Adjust PCA components based on encoding
        if encoding == 'amplitude':
            exp_config['pca_components'] = 2 ** exp_config['n_qubits']
        else:
            exp_config['pca_components'] = exp_config['n_qubits']

        # Re-fit PCA if needed
        if exp_config['pca_components'] != pca.n_components_:
            print(f"Re-fitting PCA with {exp_config['pca_components']} components...")
            pca_temp = PCA(n_components=exp_config['pca_components'])
            X_scaled = scaler.transform(X_pca)
            pca_temp.fit(X_scaled)

            # Create new dataset
            pca_dataset_temp = PCATransformedDataset(images_raw, labels_numeric, pca_temp, scaler)
            train_temp, val_temp, test_temp = random_split(
                pca_dataset_temp, [train_size, val_size, test_size],
                generator=torch.Generator().manual_seed(42)
            )
            train_loader_temp = DataLoader(train_temp, batch_size=exp_config['batch_size'], shuffle=True)
            val_loader_temp = DataLoader(val_temp, batch_size=exp_config['batch_size'], shuffle=False)
            test_loader_temp = DataLoader(test_temp, batch_size=exp_config['batch_size'], shuffle=False)
        else:
            train_loader_temp = train_loader
            val_loader_temp = val_loader
            test_loader_temp = test_loader

        # Create quantum device and circuit for this configuration
        dev_temp = qml.device('default.qubit', wires=exp_config['n_qubits'])

        if encoding == 'angle':
            @qml.qnode(dev_temp, interface='torch')
            def qcircuit_temp(inputs, weights):
                inputs = inputs.float()
                for i in range(exp_config['n_qubits']):
                    qml.RY(inputs[i], wires=i)
                if circuit_type == 'strongly_entangling':
                    qml.StronglyEntanglingLayers(weights, wires=range(exp_config['n_qubits']))
                else:
                    qml.BasicEntanglerLayers(weights, wires=range(exp_config['n_qubits']))
                return [qml.expval(qml.PauliZ(i)) for i in range(exp_config['n_qubits'])]
        else:
            @qml.qnode(dev_temp, interface='torch')
            def qcircuit_temp(inputs, weights):
                inputs = inputs.float()
                inputs_norm = inputs / torch.sqrt(torch.sum(inputs**2) + 1e-8)
                qml.AmplitudeEmbedding(features=inputs_norm, wires=range(exp_config['n_qubits']), normalize=True)
                if circuit_type == 'strongly_entangling':
                    qml.StronglyEntanglingLayers(weights, wires=range(exp_config['n_qubits']))
                else:
                    qml.BasicEntanglerLayers(weights, wires=range(exp_config['n_qubits']))
                return [qml.expval(qml.PauliZ(i)) for i in range(exp_config['n_qubits'])]

        # Create a modified model class that uses the temporary circuit
        class QuantumClassifierTemp(pl.LightningModule):
            def __init__(self, config, quantum_circuit):
                super().__init__()
                self.config = config
                self.qcircuit = quantum_circuit

                if config['circuit_type'] == 'strongly_entangling':
                    weight_shape = (config['n_layers'], config['n_qubits'], 3)
                else:
                    weight_shape = (config['n_layers'], config['n_qubits'])

                self.q_weights = nn.Parameter(torch.randn(weight_shape) * 0.1)
                self.fc = nn.Linear(config['n_qubits'], config['n_classes'])
                self.criterion = nn.CrossEntropyLoss()

            def forward(self, x):
                x = x.float()
                batch_size = x.shape[0]
                quantum_outputs = []
                for i in range(batch_size):
                    q_out = self.qcircuit(x[i].float(), self.q_weights)
                    quantum_outputs.append(torch.stack(q_out))
                quantum_outputs = torch.stack(quantum_outputs).float()
                logits = self.fc(quantum_outputs)
                return logits

            def training_step(self, batch, batch_idx):
                x, y = batch
                logits = self(x)
                loss = self.criterion(logits, y)
                preds = torch.argmax(logits, dim=1)
                acc = (preds == y).float().mean()
                self.log('train_loss', loss, prog_bar=True)
                self.log('train_acc', acc, prog_bar=True)
                return loss

            def validation_step(self, batch, batch_idx):
                x, y = batch
                logits = self(x)
                loss = self.criterion(logits, y)
                preds = torch.argmax(logits, dim=1)
                acc = (preds == y).float().mean()
                self.log('val_loss', loss, prog_bar=True)
                self.log('val_acc', acc, prog_bar=True)
                return loss

            def test_step(self, batch, batch_idx):
                x, y = batch
                logits = self(x)
                loss = self.criterion(logits, y)
                preds = torch.argmax(logits, dim=1)
                acc = (preds == y).float().mean()
                self.log('test_loss', loss)
                self.log('test_acc', acc)
                return loss

            def configure_optimizers(self):
                optimizer = torch.optim.Adam(self.parameters(), lr=self.config['learning_rate'])
                return optimizer

        # Initialize model
        model_temp = QuantumClassifierTemp(exp_config, qcircuit_temp)

        # Setup trainer
        checkpoint_callback_temp = ModelCheckpoint(
            dirpath=f'./checkpoints_comparison',
            filename=f'{encoding}-{circuit_type}-{{epoch:02d}}-{{val_acc:.4f}}',
            monitor='val_acc',
            mode='max',
            save_top_k=1
        )

        trainer_temp = pl.Trainer(
            max_epochs=exp_config['max_epochs'],
            callbacks=[checkpoint_callback_temp],
            accelerator='auto',
            devices=1,
            enable_progress_bar=True,
            enable_model_summary=False,
            logger=False
        )

        # Train and measure time
        start_time = time.time()
        trainer_temp.fit(model_temp, train_loader_temp, val_loader_temp)
        training_time = time.time() - start_time

        # Test
        test_results = trainer_temp.test(model_temp, test_loader_temp, verbose=False)

        # Store results
        result = {
            'encoding': encoding,
            'circuit_type': circuit_type,
            'test_accuracy': test_results[0]['test_acc'],
            'test_loss': test_results[0]['test_loss'],
            'training_time': training_time,
            'n_parameters': sum(p.numel() for p in model_temp.parameters()),
            'pca_components': exp_config['pca_components']
        }
        comparison_results.append(result)

        print(f"\nCompleted: Test Accuracy = {result['test_accuracy']:.4f}, Time = {training_time:.1f}s\n")

print("\n" + "="*70)
print("All experiments completed!")
print("="*70)

In [ ]:
# Create comparison DataFrame
df_results = pd.DataFrame(comparison_results)

print("\n COMPARISON RESULTS")
print("="*80)
print(df_results.to_string(index=False))
print("="*80)

In [ ]:
# Visualization 1: Accuracy Comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Bar plot: Accuracy by configuration
x_labels = [f"{r['encoding']}\n{r['circuit_type'][:6]}" for r in comparison_results]
accuracies = [r['test_accuracy'] for r in comparison_results]
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

axes[0].bar(x_labels, accuracies, color=colors, alpha=0.7, edgecolor='black')
axes[0].set_ylabel('Test Accuracy', fontsize=12, fontweight='bold')
axes[0].set_title('Test Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylim([0, 1])
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(accuracies):
    axes[0].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

# Bar plot: Training time
times = [r['training_time'] for r in comparison_results]
axes[1].bar(x_labels, times, color=colors, alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Training Time (seconds)', fontsize=12, fontweight='bold')
axes[1].set_title('Training Time Comparison', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)
for i, v in enumerate(times):
    axes[1].text(i, v + max(times)*0.02, f'{v:.1f}s', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Visualization 2: Grouped comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Group by encoding
for encoding in encodings:
    encoding_results = [r for r in comparison_results if r['encoding'] == encoding]
    circuit_names = [r['circuit_type'] for r in encoding_results]
    accs = [r['test_accuracy'] for r in encoding_results]

    idx = 0 if encoding == 'angle' else 1
    axes[0, idx].bar(circuit_names, accs, color=['#3498db', '#e74c3c'], alpha=0.7, edgecolor='black')
    axes[0, idx].set_title(f'{encoding.upper()} Encoding', fontsize=12, fontweight='bold')
    axes[0, idx].set_ylabel('Test Accuracy')
    axes[0, idx].set_ylim([0, 1])
    axes[0, idx].grid(axis='y', alpha=0.3)
    for i, v in enumerate(accs):
        axes[0, idx].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

# Group by circuit type
for circuit in circuit_types:
    circuit_results = [r for r in comparison_results if r['circuit_type'] == circuit]
    encoding_names = [r['encoding'] for r in circuit_results]
    accs = [r['test_accuracy'] for r in circuit_results]

    idx = 0 if circuit == 'strongly_entangling' else 1
    axes[1, idx].bar(encoding_names, accs, color=['#2ecc71', '#f39c12'], alpha=0.7, edgecolor='black')
    axes[1, idx].set_title(f'{circuit.replace("_", " ").title()}', fontsize=12, fontweight='bold')
    axes[1, idx].set_ylabel('Test Accuracy')
    axes[1, idx].set_ylim([0, 1])
    axes[1, idx].grid(axis='y', alpha=0.3)
    for i, v in enumerate(accs):
        axes[1, idx].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

plt.suptitle('Detailed Comparison Analysis', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

In [ ]:
# Statistical summary
print("\nSTATISTICAL SUMMARY")
print("="*80)

# Best configuration
best_result = max(comparison_results, key=lambda x: x['test_accuracy'])
print(f"\nBEST CONFIGURATION:")
print(f"   Encoding: {best_result['encoding'].upper()}")
print(f"   Circuit: {best_result['circuit_type'].upper()}")
print(f"   Test Accuracy: {best_result['test_accuracy']:.4f}")
print(f"   Training Time: {best_result['training_time']:.1f}s")

# Fastest configuration
fastest_result = min(comparison_results, key=lambda x: x['training_time'])
print(f"\nFASTEST CONFIGURATION:")
print(f"   Encoding: {fastest_result['encoding'].upper()}")
print(f"   Circuit: {fastest_result['circuit_type'].upper()}")
print(f"   Training Time: {fastest_result['training_time']:.1f}s")
print(f"   Test Accuracy: {fastest_result['test_accuracy']:.4f}")

# Encoding comparison
print(f"\nENCODING COMPARISON:")
for encoding in encodings:
    enc_results = [r for r in comparison_results if r['encoding'] == encoding]
    avg_acc = np.mean([r['test_accuracy'] for r in enc_results])
    avg_time = np.mean([r['training_time'] for r in enc_results])
    print(f"   {encoding.upper()}: Avg Accuracy = {avg_acc:.4f}, Avg Time = {avg_time:.1f}s")

# Circuit comparison
print(f"\nCIRCUIT COMPARISON:")
for circuit in circuit_types:
    circ_results = [r for r in comparison_results if r['circuit_type'] == circuit]
    avg_acc = np.mean([r['test_accuracy'] for r in circ_results])
    avg_time = np.mean([r['training_time'] for r in circ_results])
    print(f"   {circuit.upper()}: Avg Accuracy = {avg_acc:.4f}, Avg Time = {avg_time:.1f}s")

print("\n" + "="*80)

In [ ]:
# Save comparison results
df_results.to_csv('/mnt/uifr-data/outputs/pca_comparison_results.csv', index=False)
print("\n Comparison results saved to: pca_comparison_results.csv")

## Summary and Next Steps

This notebook demonstrated:
1.  PCA-baifd dimensionality reduction for quantum-compatible data
2.  Angle encoding and amplitude encoding options
3.  Strongly entangling vs basic entangling circuits
4.  Full quantum variational classifier with PyTorch Lightning

**Next Steps:**
- Try different numbers of qubits and layers
- Exforiment with different circuit architectures
- Compare forformance between encoding methods
- Try hybrid quantum-classical approaches (ife next notebook)

In [ ]:
# Save the trained model
torch.save({
    'model_state_dict': model.state_dict(),
    'withfig': CONFIG,
    'pca': pca,
    'scaler': scaler
}, 'quantum_classifier_pca.pth')

print("Model saved successfully!")